In [1]:
from sklearn.base import BaseEstimator # BaseEstimator가 부모?

In [2]:
### 클래스 만들기
class MyDummyClassifier(BaseEstimator): # BaseEstimator에서 상속받아 만듦
    def fit(self,X,y=None):  
        pass
    def predict(self,X):
        import numpy as np
        pred = np.zeros((X.shape[0],1))  # 행렬을 만드는데 0으로 채워진 것을 만든다. () 안에 몇 행 몇 열인지 적음
        # 행단위로 데이터가 2건이 들어오면 2행이, 열은 1개
        for i in range(X.shape[0]): # 데이터 건수만큼 반복
            if X['Sex'].iloc[i] == 1: 
                pred[i] = 0 # 학습시키지 않고 데이터 보고 남자인 경우 사망?
            else:
                pred[i] = 1
        return pred

In [3]:
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df
def drop_features(df):
    df.drop(columns = ['PassengerId','Name','Ticket'],inplace=True) # 필요없는 열 제거
    return df
def format_features(df):
    from sklearn.preprocessing import LabelEncoder
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin','Sex','Embarked']
    for feature in features: # 각 열의 레이블을 만들고 학습
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature]) # 변경된 것을 원래 위치에 넣기
        print(le.classes_)
    return df
def transform_features():
    import pandas as pd
    df = pd.read_csv('titanic_train.csv')
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [4]:
titanic_df = transform_features() # 데이터 가져오기

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
['female' 'male']
['C' 'N' 'Q' 'S']


In [5]:
# x값 y값 분리
from sklearn.model_selection import train_test_split
X_df = titanic_df.drop(columns=['Survived'])
y_df = titanic_df['Survived']
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=11)

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
### 정확도 구하기
# 만들어둔 모델에 넣고 확인
my = MyDummyClassifier()
my.fit(X_train,y_train)
pred = my.predict(X_test)
accuracy_score(y_test,pred)
# 성별에 따라 생존여부를 나누었을 때 정확도가 83.24% 
# -> 정확도만으로는 모델이 학습이 잘 되었는지 판단하기가 어려움

0.8324022346368715

In [8]:
### 오차 행렬 confusion_matrix
# 0쪽이 아닌 1쪽을 기준으로 
from sklearn.metrics import confusion_matrix

In [9]:
confusion_matrix(y_test,pred) # 103, 46 맞힘

array([[103,  15],
       [ 15,  46]], dtype=int64)

In [10]:
# 정확도 지표는 비대칭한 데이터 세트에서 Positive에 대한 예측 정확도를 판단하지 못한 채 Negative에 대한 
# 예측 정확도만으로도 분류의 정확도가 맴우 높게 나타나는 수치적인 판단 오류를 일으키게 됨
# 데이터가 균형이 잡혀있으면 정확도 지표도 괜찮지만 불균형한 경우 문제가 생길 수 있음

# 정밀도 : 예측값이 1인 것중에 맞힌 값의 비율
# 재현율 : 실제값이 1인 것중에 맞힌 값의 비율

In [11]:
### 정밀도와 재현율 구하기
# 정밀도 : 46/(15+46)
# 재현도(민감도, tpr) : 46/(15+46)
# 정밀도와 재현율은 둘 다 높을수록 좋지만 그러기 어렵기 때문에 이진 분류 모델의 업무 특성에 따라 특정 평가 지표가 더 중요한
# 지표로 간주될 수 있음. 재현율이 중요지표인 경우 실제 Positive 양성 데이터를 Negative로 잘못 판단했을 때 업무상 큰 영향이 발생하는 경우
# 예) 암 판단 모델, 보험사기 적발 모델은 재현율이 중요
# 실제 긴데 아니라고 하면 심각해짐 -> 재현율 중요
# 실제 아닌데 기라고 하면 심각해짐 -> 정밀도 중요

In [12]:
from sklearn.metrics import recall_score,precision_score

In [13]:
### 1) 재현율 구하기
recall_score(y_test,pred)

0.7540983606557377

In [14]:
### 2) 정밀도 구하기
precision_score(y_test,pred)

0.7540983606557377

In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score

In [16]:
# 함수로 만들기
def get_clf_eval(y_test,pred):
    from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score
    accuracy = accuracy_score(y_test,pred)
    confusion = confusion_matrix(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print(f'정확도: {accuracy:.4f} 정밀도: {precision:.4f} 재현율: {recall:.4f}')

In [17]:
# 제대로 동작하는지 확인
get_clf_eval(y_test,pred)

오차행렬
[[103  15]
 [ 15  46]]
정확도: 0.8324 정밀도: 0.7541 재현율: 0.7541


In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
titanic_df = transform_features()
from sklearn.model_selection import train_test_split
X_df = titanic_df.drop(columns=['Survived'])
y_df = titanic_df['Survived']
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=11)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
['female' 'male']
['C' 'N' 'Q' 'S']


In [20]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train,y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test,pred)

오차행렬
[[108  10]
 [ 14  47]]
정확도: 0.8659 정밀도: 0.8246 재현율: 0.7705


In [21]:
pred_proba = lr_clf.predict_proba(X_test)

In [23]:
pred_proba.shape  # 데이터 179개, 열 2개

(179, 2)

In [26]:
pred[:3] # 1차원

array([1, 0, 0], dtype=int64)

In [25]:
pred_proba [:3] # 2차원
# 0일 확률 0.44935228 1일 확률 0.55064772이니 결정값 1
# 1일 될 확률이 0.5보다 크면 1이됨
# 3개만 보기 위해 [:3]

array([[0.44935228, 0.55064772],
       [0.86335513, 0.13664487],
       [0.86429646, 0.13570354]])

In [27]:
import numpy as np

In [30]:
pred_proba_result = np.concatenate([pred_proba,pred.reshape(-1,1)],axis=1) # 컬럼 단위로 붙이기 위해 axis=1
pred_proba_result[:3]  # [0일확률, 1일확률, 결정값]

array([[0.44935228, 0.55064772, 1.        ],
       [0.86335513, 0.13664487, 0.        ],
       [0.86429646, 0.13570354, 0.        ]])

In [31]:
from sklearn.preprocessing import Binarizer

In [32]:
X=[[1,-1,2],
   [2,0,0],
   [0,1.1,1.2]]  # 2차원의 리스트형태

In [35]:
bi = Binarizer()

In [36]:
bi.fit_transform(X)  # 임계값을 0.0를 기준으로 0.0까지는 0이고 그것보다 크면 1

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 1.]])

In [37]:
bi = Binarizer(threshold=1.1)  # 임계값을 1.1을 기준으로 할 때

In [38]:
bi.fit_transform(X)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [42]:
custom_threshold = 0.5
pred_proba_1 = pred_proba[:,1].reshape(-1,1) 
# 행은 전부, 열은 둘 중 1일 확률인 뒤의 것만 가져와도 구분됨
# 1차원을 2차원으로 만들기 위해 reshape
bi = Binarizer(threshold=custom_threshold)
custom_predict = bi.fit_transform(pred_proba_1)

In [43]:
get_clf_eval(y_test,custom_predict)
# 오차행렬
# [[108  10]
#  [ 14  47]]
# 정확도: 0.8659 정밀도: 0.8246 재현율: 0.7705

# 임계치를 0.5로 했구나

오차행렬
[[108  10]
 [ 14  47]]
정확도: 0.8659 정밀도: 0.8246 재현율: 0.7705


In [45]:
custom_threshold = 0.4  # 임계치 줄임 -> 1로 예측될 확률이 높아져 재현율 상승, 정밀동 하락(반대방향으로 움직임)
pred_proba_1 = pred_proba[:,1].reshape(-1,1) 
# 행은 전부, 열은 둘 중 1일 확률인 뒤의 것만 가져와도 구분됨
# 1차원을 2차원으로 만들기 위해 reshape
bi = Binarizer(threshold=custom_threshold)
custom_predict = bi.fit_transform(pred_proba_1)
get_clf_eval(y_test,custom_predict)

오차행렬
[[97 21]
 [11 50]]
정확도: 0.8212 정밀도: 0.7042 재현율: 0.8197


In [48]:
thresholds = [0.4,0.45,0.5,0.55,0.6]
def get_eval_by_threshold(y_test,pred_proba_c1,thresholds): # 1이라고 예측한 확률 pred_proba_c1
    for threshold in thresholds:
        bi = Binarizer(threshold=threshold)
        custom_predict = bi.fit_transform(pred_proba_c1)
        print('임계값 :', threshold)
        get_clf_eval(y_test,custom_predict)

In [49]:
get_eval_by_threshold(y_test,pred_proba_1,thresholds)

임계값 : 0.4
오차행렬
[[97 21]
 [11 50]]
정확도: 0.8212 정밀도: 0.7042 재현율: 0.8197
임계값 : 0.45
오차행렬
[[105  13]
 [ 13  48]]
정확도: 0.8547 정밀도: 0.7869 재현율: 0.7869
임계값 : 0.5
오차행렬
[[108  10]
 [ 14  47]]
정확도: 0.8659 정밀도: 0.8246 재현율: 0.7705
임계값 : 0.55
오차행렬
[[111   7]
 [ 16  45]]
정확도: 0.8715 정밀도: 0.8654 재현율: 0.7377
임계값 : 0.6
오차행렬
[[113   5]
 [ 17  44]]
정확도: 0.8771 정밀도: 0.8980 재현율: 0.7213


In [ ]:
# 0.45로 하는 것이 0.5로 하는 것이 더 좋음 -> 정밀도와 재현율이 같이